In [25]:
import dask
import dask.dataframe as ddf
import pandas as pd
import numpy as np
import ast
import collections
import math
import re
import pickle

In [2]:
parquetdir = '../../tcm-columns-add-main/'

In [3]:
# revision_text= ddf.read_parquet(parquetdir + 'revision.text').compute()

In [4]:
page_title = ddf.read_parquet(parquetdir + 'page.title').compute()

In [5]:
links = pd.read_parquet('../../intermediate-result/TCM/wikilink-info')

In [6]:
links['wikilinks'] = links['wikilinks'].map(lambda x: ast.literal_eval(x.lower()))

In [7]:
page_texts = page_title.join(links)

In [8]:
def get_link_title(links):
    return [l[2:-2].split('|')[0] for l in links]

In [9]:
# count the number of apperance for each term in each page

In [10]:
page_texts['wikilinks'] = page_texts['wikilinks'].apply(get_link_title)

In [11]:
# # take the set so each link appears only once in each revision
# page_texts['wikilinks'] = page_texts['wikilinks'].map(lambda x: list(set(x)))

In [12]:
x = page_texts.groupby('page.title').agg({'wikilinks':'sum'})

In [13]:
x['wikilinks'] = x['wikilinks'].map(lambda x : collections.Counter(x))

In [14]:
month = ['january','february','march','april','may','june','july','august','september','october','november','december']

In [15]:
def filter_links(counter):
    removed_keys = [i for i in counter if re.match(r'^([\s\d]+)$', i) or any(xs in i for xs in month) or re.match('\d{4}[-/]\d{2}[-/]\d{2}',i)]
    for i in removed_keys:
        del counter[i]
    return counter

In [16]:
a = x['wikilinks'].apply(filter_links)

In [17]:
res = {}

In [18]:
def count_links(df):
    links = df['wikilinks']
    title = df.name
    for k,v in links.items():
        if k not in res:
            res[k] = {}
        res[k][title] = v

In [19]:
b = x.apply(count_links,axis=1)

In [21]:
with open('links-count.pkl', 'wb') as f:
    pickle.dump(res, f)

In [6]:
# with open('links-count.pkl', 'rb') as f:
#     x = pickle.load(f)

# df idf

In [23]:
res1 = {}

In [24]:
def insert_into_dict(d):
    for k,v in d.items():
#         if k not in kept_links:
#             continue
        if k not in res1:
            res1[k] = v
        else:
            for k1,v1 in v.items():
                res1[k][k1] = v1

In [25]:
insert_into_dict(res)

In [26]:
# document frequency of each term
df_t = {}

In [27]:
for k,v in res.items():
    df_t[k] = len(v)

In [29]:
df = pd.DataFrame.from_dict(df_t, orient="index").rename(columns={0:'df'})

In [30]:
N = len(df)

In [31]:
df['idf'] = df['df'].map(lambda x: math.log(N / x + 1))

In [32]:
df.to_parquet('df-idf-for-each-term')

In [33]:
df1 = pd.DataFrame.from_dict(res, orient="index").stack().to_frame()

In [34]:
df1 = df1.rename(columns={0:'tf'}) 

In [35]:
df1 = df1.reset_index()

In [36]:
df1 = df1.rename(columns={'level_0':'wikilink','level_1':'page.title'})

In [37]:
df = df1.set_index('wikilink').join(df)

In [38]:
df['wf'] = df['tf'].map(lambda x: 1 + math.log(x + 1) if x > 0 else 0)

In [39]:
df = df[df.index!='']

In [40]:
df['wf-idf'] = df['wf'] * df['idf']

In [41]:
res = df[['page.title','wf-idf']]

In [42]:
res.to_parquet('weight-for-each-term-each-page')

In [43]:
df = res.reset_index().rename(columns={'index':'link'})

In [44]:
def cosine_similarity(counter1,counter2):
    c1squared= sum([a*a for a in counter1.values()])
    c2squared= sum([a*a for a in counter2.values()])
    c1c2=sum([counter1[k]*counter2[k]  for k in counter1.keys() if k in counter2])
    return c1c2/(math.sqrt(c1squared*c2squared))

In [45]:
def get_links(x):
    return x.set_index('link')[['wf-idf']].to_dict()['wf-idf']

In [46]:
page_links = df.groupby('page.title')

In [47]:
page_wlinkcounts = {}

In [48]:
for page in df['page.title'].unique():
    page_wlinkcounts[page] = get_links(page_links.get_group(page))

In [49]:
with open('page-wikicount-wf-idf.pkl', 'wb') as f:
    pickle.dump(page_wlinkcounts, f)

In [50]:
page_wlink_similarities={
    key1:{
        k:v for k,v in {
            key2: cosine_similarity(page_wlinkcounts[key1],page_wlinkcounts[key2]) 
            for key2 in page_wlinkcounts.keys() 
            if not key1.startswith('Talk:') and not key2.startswith('Talk:') 
                 and page_wlinkcounts[key1] and page_wlinkcounts[key2]
            }.items() if 1>v>0.001 
        } for key1 in page_wlinkcounts.keys()}


In [51]:
with open('article-similarity-wf-idf.pkl', 'wb') as f:
    pickle.dump(page_wlink_similarities, f)

In [52]:
df = pd.DataFrame.from_dict(page_wlink_similarities,orient='index')
df = df.fillna(0)
df = df[df.index.to_list()]

In [53]:
np.fill_diagonal(df.values,1)

In [54]:
df.to_csv("article-similarity-matrix-wf-idf.csv", sep="\t",encoding='utf-16')

In [55]:
df.to_parquet('article-similarity-matrix-wf-idf')